# Basketball Arena Demand Prediction Model

This notebook implements a multi-output regression model to predict attendance demand across four seating sections:
- Bleachers
- Lower Tier
- Courtside
- Luxury Boxes

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seed for reproducibility
seed = 42
np.random.seed(seed)

## 1. Load Data

Load the processed data from your arena_analysis notebook or data source.

In [ ]:
# Load the preprocessed X and y data from CSV files
print("Loading data from CSV files...")

# Load feature matrix (X)
X = pd.read_csv("X_league.rs_games.csv")
print(f"Features (X) shape: {X.shape}")

# Load target matrix (y)
y = pd.read_csv("y_league.rs_games.csv")
print(f"Targets (y) shape: {y.shape}")

print("\nFeature columns:")
print(list(X.columns))

print("\nTarget columns:")
print(list(y.columns))

print("\nData loaded successfully!")

In [ ]:
# calculate losses, win frequencies, and points diff per game
X['home_losses'] = (X['round_number'] - 1) - X['home_wins']
X['away_losses'] = (X['round_number'] - 1) - X['away_wins']

# win frequencies
X['home_win_freq'] = X.apply(lambda row: row['home_wins'] / (row['round_number'] - 1) if (row['round_number'] - 1) > 0 else 0, axis=1)
X['away_win_freq'] = X.apply(lambda row: row['away_wins'] / (row['round_number'] - 1) if (row['round_number'] - 1) > 0 else 0, axis=1)

# points diff per game
X['home_points_diff_per_game'] = X.apply(lambda row: row['home_points_diff'] / (row['round_number'] - 1) if (row['round_number'] - 1) > 0 else 0, axis=1)
X['away_points_diff_per_game'] = X.apply(lambda row: row['away_points_diff'] / (row['round_number'] - 1) if (row['round_number'] - 1) > 0 else 0, axis=1)

## 2. Define Features and Targets

In [ ]:
# Feature columns for demand prediction
feature_cols = [
    'league_level',
    'round_number',
    'home_win_freq',
    'home_points_diff_per_game',
    'away_win_freq',
    'away_points_diff_per_game',
    'home_prev_level_diff',
    'away_prev_level_diff',
    # 'home_prev_season_wins',
    # 'away_prev_season_wins',
    'home_won_prev_game',
    'away_won_prev_game',
    'bleachers_price',
    'lower_tier_price',
    'courtside_price',
    'luxury_boxes_price',
    'is_tv_game'
]

# Target columns (attendance for each seating section)
target_cols = [
    'bleachers_attendance',
    'lower_tier_attendance',
    'courtside_attendance',
    'luxury_boxes_attendance'
]

print(f"Features: {len(feature_cols)} columns")
print(f"Targets: {len(target_cols)} columns")

# keep only the relevant columns in X and y
X = X[feature_cols]
y = y[target_cols]

## 3. Data Exploration and Preparation

In [ ]:
# Data exploration and preparation
print("Dataset shapes:")
print(f"X (features): {X.shape}")
print(f"y (targets): {y.shape}")

print("\nChecking for missing values:")
print("X missing values:", X.isnull().sum().sum())
print("y missing values:", y.isnull().sum().sum())

print("\nFeature statistics:")
print(X.describe())

print("\nTarget statistics:")
print(y.describe())

# Check for any infinite or extremely large values
print("\nChecking for infinite values:")
for col in X.columns:
    if X[col].dtype in ['float64', 'int64']:
        if np.isinf(X[col]).any():
            print(f"Infinite values found in {col}")

for col in y.columns:
    if y[col].dtype in ['float64', 'int64']:
        if np.isinf(y[col]).any():
            print(f"Infinite values found in {col}")

# check if any columns in X or y have null or NaN values
print("\nChecking for NaN values:")
for col in X.columns:
    if X[col].isnull().any():
        print(f"NaN values found in {col}")
# replace Nan in home_won_prev_game with True
X.loc[X['home_won_prev_game'].isnull(), 'home_won_prev_game'] = True
X.loc[X['away_won_prev_game'].isnull(), 'away_won_prev_game'] = True
# Remove any rows with missing values
initial_rows = X.shape[0]
mask = ~(X.isnull().any(axis=1) | y.isnull().any(axis=1))
X = X[mask]
y = y[mask]

print(f"\nRows before cleaning: {initial_rows}")
print(f"Rows after cleaning: {X.shape[0]}")
print(f"Rows removed: {initial_rows - X.shape[0]}")

## 4. Train/Test Split

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=seed,
    shuffle=True,
    stratify=X['league_level'] if 'league_level' in X.columns else None
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Features: {X_train.shape[1]}")
print(f"Targets: {y_train.shape[1]}")

## 5. Model Pipeline Setup

In [ ]:
# Create a pipeline with preprocessing and model
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Normalize numeric features
    ('rf', RandomForestRegressor(
        n_estimators=100,
        max_depth=10,
        random_state=42,
        n_jobs=-1
    ))
])

print("Pipeline created successfully")

## 6. Hyperparameter Tuning

In [ ]:
# Define parameter grid for hyperparameter tuning
param_grid = {
    'rf__n_estimators': [50, 250, 300, 350, 500],
    'rf__max_depth': [6, 13, None],
    'rf__min_samples_leaf': [1, 10, 15],
    'rf__min_samples_split': [2, 5, 10]
}

# Set up GridSearchCV
grid = GridSearchCV(
    pipeline,
    param_grid,
    cv=4,
    scoring='neg_mean_squared_error',
    verbose=3,
    n_jobs=-1
)

print("GridSearchCV setup complete")
print(f"Total parameter combinations: {len(param_grid['rf__n_estimators']) * len(param_grid['rf__max_depth']) * len(param_grid['rf__min_samples_leaf']) * len(param_grid['rf__min_samples_split'])}")

In [ ]:
# Uncomment to run hyperparameter tuning (this will take some time)
# 
print("Starting hyperparameter tuning...")
grid.fit(X_train, y_train)

print("\nBest parameters:")
for param, value in grid.best_params_.items():
    print(f"  {param}: {value}")

print(f"\nScoring metric: {grid.scoring}")
print(f"Best cross-validation score: {-grid.best_score_:.3f} {grid.scoring}")
# Get the best model
model = grid.best_estimator_

## 7. Alternative: Quick Model Training (Skip Hyperparameter Tuning)

In [ ]:
# # Train the model with default parameters (faster than hyperparameter tuning)
# print("Training model with default parameters...")
# model = pipeline
# model.fit(X_train, y_train)
# print("Model training complete!")

## 8. Model Evaluation

In [ ]:
# Evaluate the model
print("Making predictions on test set...")
y_pred = model.predict(X_test)

# Convert predictions to DataFrame for easier handling
y_pred_df = pd.DataFrame(y_pred, columns=y.columns, index=y_test.index)

print("Model Evaluation Results:")
print("=" * 50)

# Calculate metrics for each seating section
results = []
for i, col in enumerate(y.columns):
    rmse = root_mean_squared_error(y_test.iloc[:,i], y_pred[:,i])
    r2 = r2_score(y_test.iloc[:,i], y_pred[:,i])
    mae = np.mean(np.abs(y_test.iloc[:,i] - y_pred[:,i]))
    
    results.append({
        'Section': col.replace('_attendance', '').replace('_', ' ').title(),
        'RMSE': rmse,
        'R²': r2,
        'MAE': mae
    })
    
    print(f"{col.replace('_attendance', '').replace('_', ' ').title():15} - RMSE: {rmse:6.1f} seats, R²: {r2:5.3f}, MAE: {mae:6.1f} seats")

# Overall metrics
overall_rmse = np.sqrt(mean_squared_error(y_test.values.flatten(), y_pred.flatten()))
print(f"\nOverall RMSE: {overall_rmse:.1f} seats")

# Create results DataFrame
results_df = pd.DataFrame(results)
print("\nDetailed Results:")
print(results_df.round(3))

## 9. Feature Importance Analysis

In [ ]:
# Analyze feature importance
print("Analyzing feature importance...")

# Get feature importance from the trained Random Forest
feature_importance = model.named_steps['rf'].feature_importances_

# Create feature importance DataFrame
importance_df = pd.DataFrame({
    'feature': X.columns,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

print("Feature Importance Rankings:")
print("=" * 40)
for idx, row in importance_df.iterrows():
    print(f"{row['feature']:25} {row['importance']:6.3f}")

# Plot feature importance
plt.figure(figsize=(10, 8))
sns.barplot(data=importance_df.head(10), x='importance', y='feature')
plt.title('Top 10 Most Important Features for Attendance Prediction')
plt.xlabel('Feature Importance')
plt.tight_layout()
plt.show()

## 10. Prediction Visualization

In [ ]:
# Create prediction visualizations
print("Creating prediction visualizations...")

# starting capacity of arena sections
init_capacity = {
    'bleachers': 5438,
    'lower tier': 500,
    'courtside': 60,
    'luxury boxes': 2,
}
# Create subplots for each seating section
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.flatten()

for i, col in enumerate(y.columns):
    ax = axes[i]
    
    # Scatter plot of actual vs predicted
    ax.scatter(y_test.iloc[:,i], y_pred[:,i], alpha=0.6)

    # plot also train in red
    y_train_pred = model.predict(X_train)
    ax.scatter(y_train.iloc[:,i], y_train_pred[:,i], alpha=0.3, color='red', label='Train Predictions')
    
    # Perfect prediction line
    min_val = min(y_test.iloc[:,i].min(), y_pred[:,i].min())
    max_val = max(y_test.iloc[:,i].max(), y_pred[:,i].max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2)
    
    # Labels and title
    section_name = col.replace('_attendance', '').replace('_', ' ').title()
    print(f"Plotting {section_name} attendance...")
    ax.set_xlabel(f'Actual {section_name} Attendance')
    ax.set_ylabel(f'Predicted {section_name} Attendance')
    ax.set_title(f'{section_name} - Actual vs Predicted')
    ax.grid(True, linestyle='--', alpha=0.7)

    # show vertical line at initial capacity
    if section_name.replace('_attendance', '').lower() in init_capacity:
        init_cap = init_capacity[section_name.replace('_attendance', '').lower()]
        ax.axvline(x=init_cap, color='orange', linestyle='--', label='Initial Capacity')
        # ax.axhline(y=init_cap, color='orange', linestyle='--')
    
    # Add R² score to plot
    r2 = r2_score(y_test.iloc[:,i], y_pred[:,i])
    ax.text(0.05, 0.95, f'R² = {r2:.3f}', transform=ax.transAxes, 
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

# Outliers:

In [ ]:
# show the worst predictions
seat_types = [1,]
worst_predictions = pd.DataFrame({
    'Actual': y_test.values[:,seat_types].flatten(),
    'Predicted': y_pred[:,seat_types].flatten(),
    'Error': y_test.values[:,seat_types].flatten() - y_pred[:,seat_types].flatten()
})
print(f"\nWorst Predictions (Top 10 by Error in {seat_types}):")
worst_predictions = worst_predictions.sort_values(by='Error', ascending=False).head(10)
print(worst_predictions)

## 11. Prediction Function

In [ ]:
def predict_attendance(model, game_features):
    """
    Predict attendance for a new game given its features.
    
    Parameters:
    model: Trained sklearn model
    game_features: dict or DataFrame with feature values
    
    Returns:
    Dictionary with predicted attendance for each section
    """
    # Convert to DataFrame if it's a dictionary
    if isinstance(game_features, dict):
        game_df = pd.DataFrame([game_features])
    else:
        game_df = game_features.copy()
    
    # Ensure all required features are present
    missing_features = set(X.columns) - set(game_df.columns)
    if missing_features:
        raise ValueError(f"Missing features: {missing_features}")
    
    # Select only the required features in the correct order
    game_df = game_df[X.columns]
    
    # Make prediction
    prediction = model.predict(game_df)
    
    # Return as dictionary
    result = {}
    for i, col in enumerate(y.columns):
        section_name = col.replace('_attendance', '')
        result[section_name] = int(round(prediction[0][i]))
    
    return result

print("Prediction function defined")

## 12. Example Prediction

## 13. Model Persistence

In [ ]:
# Save the trained model for future use
# Uncomment when you have a trained model
# 
import joblib

# # Save the model
# model_filename = 'arena_demand_prediction_model.pkl'
# joblib.dump(model, model_filename)
# print(f"Model saved as {model_filename}")

# To load the model later:
loaded_model = joblib.load('arena_demand_prediction_model.pkl')
model = loaded_model

In [ ]:
def get_revenue(prices, attendance):
    """
    Calculate total revenue based on prices and attendance.
    
    Parameters:
    prices: dict with section prices
    attendance: dict with predicted attendance for each section
    
    Returns:
    Total revenue as float
    """
    section_mapping = {
        'bleachers_price': 'bleachers',
        'lower_tier_price': 'lower_tier',
        'courtside_price': 'courtside',
        'luxury_boxes_price': 'luxury_boxes'
    }
    total_revenue = 0.0
    for section, price in prices.items():
        section_name = section_mapping.get(section, section)
        section_revenue = price * attendance.get(section_name, 0)
        print(f"Calculating revenue for {section_name}: ${price} * {attendance.get(section_name, 0)} = ${section_revenue:.2f}")
        total_revenue += section_revenue
    return total_revenue if total_revenue >= 0 else 0.0  #
    

# Example of how to make a prediction for a new game
# Uncomment and modify with actual values when ready to use
# 
# # Example game features
# Example base game features (modify these for your specific scenario)
input_prices = {
    'bleachers_price': 8,
    'lower_tier_price': 30,
    'courtside_price': 80,
    'luxury_boxes_price': 420
}
    
example_game = {
    'league_level': 3,
    'round_number': 4,
    'home_win_freq': 1,
    'home_points_diff_per_game': 65/3,
    'away_win_freq': 0,
    'away_points_diff_per_game': -42/3,
    'home_prev_level_diff': 0,
    'away_prev_level_diff': 0,
    'home_prev_season_wins': 7,
    'away_prev_season_wins': 13,
    'home_won_prev_game': True,
    'away_won_prev_game': False,
    'is_tv_game': False,
    **input_prices
}

# Make prediction
predicted_attendance = predict_attendance(model, example_game)
print(predicted_attendance)
print("Predicted attendance for example game:")
print("=" * 40)
for section, attendance in predicted_attendance.items():
    print(f"{section.replace('_', ' ').title():15}: {attendance:4d} seats")

total_predicted = sum(predicted_attendance.values())
print(f"\nTotal predicted attendance: {total_predicted:,} seats")



print("\nCalculating revenue for predicted attendance...")
print(f"Revenue for predicted attendance: ${get_revenue(input_prices, predicted_attendance):,.2f}")



## 14. Price Optimization Predictor

Find the optimal pricing strategy by testing all possible integer price combinations within 2x-4x ranges and maximizing total revenue.

In [ ]:
# Example base game features (modify these for your specific scenario)
base_game_features = {
    'league_level': 3,
    'round_number': 4,
    'home_win_freq': 1,
    'home_points_diff_per_game': 65/3,
    'away_win_freq': 0,
    'away_points_diff_per_game': -42/3,
    'home_prev_level_diff': 0,
    'away_prev_level_diff': 0,
    'home_prev_season_wins': 7,
    'away_prev_season_wins': 13,
    'home_won_prev_game': True,
    'away_won_prev_game': False,
    'is_tv_game': False,
}

In [ ]:
# Run price optimization with reduced ranges for demonstration
# (Using smaller ranges to make computation feasible)

price_ranges = {
    'bleachers': (5, 12),
    'lower_tier': (18, 70),
    'courtside': (50, 200),
    'luxury_boxes': (400, 1600) # Original max range
}

# no limits for this example
capacity_limits = {
    'bleachers': np.inf,  # No limit for bleachers
    'lower_tier': np.inf,  # No limit for lower tier
    'courtside': np.inf,  # No limit for courtside
    'luxury_boxes': np.inf  # No limit for luxury boxes
}

In [ ]:
# Improved price optimization with luxury box increments of 100
def optimize_pricing_fast(model, base_game_features, price_ranges, capacity_limits):
    """
    Fast price optimization with luxury boxes in increments of 100.
    """
    import itertools
    from tqdm import tqdm

    # Generate price ranges with luxury boxes in increments of 100
    increments = {
        'bleachers': 1,
        'lower_tier': 2,
        'courtside': 15,
        'luxury_boxes': 100
    }
    bleacher_prices = range(price_ranges['bleachers'][0], price_ranges['bleachers'][1] + 1, increments['bleachers'])
    lower_tier_prices = range(price_ranges['lower_tier'][0], price_ranges['lower_tier'][1] + 1, increments['lower_tier'])
    courtside_prices = range(price_ranges['courtside'][0], price_ranges['courtside'][1] + 1, increments['courtside'])
    luxury_prices = range(price_ranges['luxury_boxes'][0], price_ranges['luxury_boxes'][1] + 1, increments['luxury_boxes'])

    total_combinations = len(bleacher_prices) * len(lower_tier_prices) * len(courtside_prices) * len(luxury_prices)
    print(f"Testing {total_combinations:,} price combinations (increments of {increments.values()})")

    max_revenue = 0
    optimal_prices = None
    attendance_at_optimum = None
    
    # Test all combinations
    for bleacher_p, lower_p, courtside_p, luxury_p in tqdm(
        itertools.product(bleacher_prices, lower_tier_prices, courtside_prices, luxury_prices),
        total=total_combinations,
        desc="Optimizing prices"
    ):
        # Create game features with current prices
        game_features = base_game_features.copy()
        game_features.update({
            'bleachers_price': float(bleacher_p),
            'lower_tier_price': float(lower_p),
            'courtside_price': float(courtside_p),
            'luxury_boxes_price': float(luxury_p)
        })
        
        # Predict attendance
        predicted_attendance = predict_attendance(model, game_features)
        
        # Apply capacity constraints
        constrained_attendance = {
            'bleachers': min(predicted_attendance['bleachers'], capacity_limits['bleachers']),
            'lower_tier': min(predicted_attendance['lower_tier'], capacity_limits['lower_tier']),
            'courtside': min(predicted_attendance['courtside'], capacity_limits['courtside']),
            'luxury_boxes': min(predicted_attendance['luxury_boxes'], capacity_limits['luxury_boxes'])
        }
        
        # Calculate total revenue
        revenue = (
            constrained_attendance['bleachers'] * bleacher_p +
            constrained_attendance['lower_tier'] * lower_p +
            constrained_attendance['courtside'] * courtside_p +
            constrained_attendance['luxury_boxes'] * luxury_p
        )
        
        # Update optimal if this is better
        if revenue > max_revenue:
            max_revenue = revenue
            optimal_prices = {
                'bleachers_price': bleacher_p,
                'lower_tier_price': lower_p,
                'courtside_price': courtside_p,
                'luxury_boxes_price': luxury_p
            }
            attendance_at_optimum = constrained_attendance
            print(f"New optimal found: ${max_revenue:,.2f} with prices {optimal_prices}")
            print(f"Attendance: {attendance_at_optimum}")
    
    return {
        'optimal_prices': optimal_prices,
        'optimal_attendance': attendance_at_optimum,
        'max_revenue': max_revenue,
        'total_combinations_tested': total_combinations
    }

In [ ]:
# Alternative: Use random search or simulated annealing instead of gradients
def optimize_pricing_random_search(model, base_game_features, price_ranges, capacity_limits, 
                                  n_iterations=10000, initial_temp=1000, cooling_rate=0.995):
    """
    Use simulated annealing for optimization - better for discontinuous functions.
    """
    import numpy as np
    from tqdm import tqdm
    
    def calculate_revenue(prices):
        """Calculate revenue for given prices"""
        game_features = base_game_features.copy()
        game_features.update(prices)
        
        predicted_attendance = predict_attendance(model, game_features)
        
        # Apply capacity constraints
        constrained_attendance = {
            'bleachers': min(predicted_attendance['bleachers'], capacity_limits['bleachers']),
            'lower_tier': min(predicted_attendance['lower_tier'], capacity_limits['lower_tier']),
            'courtside': min(predicted_attendance['courtside'], capacity_limits['courtside']),
            'luxury_boxes': min(predicted_attendance['luxury_boxes'], capacity_limits['luxury_boxes'])
        }
        
        revenue = (
            constrained_attendance['bleachers'] * prices['bleachers_price'] +
            constrained_attendance['lower_tier'] * prices['lower_tier_price'] +
            constrained_attendance['courtside'] * prices['courtside_price'] +
            constrained_attendance['luxury_boxes'] * prices['luxury_boxes_price']
        )
        
        return revenue, constrained_attendance
    
    # Initialize with random prices
    current_prices = {
        'bleachers_price': np.random.uniform(*price_ranges['bleachers']),
        'lower_tier_price': np.random.uniform(*price_ranges['lower_tier']),
        'courtside_price': np.random.uniform(*price_ranges['courtside']),
        'luxury_boxes_price': np.random.uniform(*price_ranges['luxury_boxes'])
    }
    
    current_revenue, current_attendance = calculate_revenue(current_prices)
    
    best_prices = current_prices.copy()
    best_revenue = current_revenue
    best_attendance = current_attendance
    
    temperature = initial_temp
    revenue_history = []
    
    print(f"Starting simulated annealing optimization...")
    print(f"Initial prices: {current_prices}")
    print(f"Initial revenue: ${current_revenue:,.2f}")
    
    for iteration in tqdm(range(n_iterations), desc="Simulated annealing"):
        # Generate neighbor solution
        new_prices = {}
        for price_key in current_prices.keys():
            section = price_key.replace('_price', '')
            min_price, max_price = price_ranges[section]
            
            # Random perturbation proportional to temperature
            perturbation = np.random.normal(0, temperature * 0.01)
            new_price = current_prices[price_key] + perturbation
            new_price = np.clip(new_price, min_price, max_price)
            new_prices[price_key] = new_price
        
        # Evaluate new solution
        new_revenue, new_attendance = calculate_revenue(new_prices)
        
        # Accept or reject based on simulated annealing criteria
        delta_revenue = new_revenue - current_revenue
        
        if delta_revenue > 0 or np.random.random() < np.exp(delta_revenue / temperature):
            current_prices = new_prices
            current_revenue = new_revenue
            current_attendance = new_attendance
            
            # Update best if this is better
            if new_revenue > best_revenue:
                best_prices = new_prices.copy()
                best_revenue = new_revenue
                best_attendance = new_attendance
                print(f"New best at iteration {iteration}: ${best_revenue:,.2f}")
        
        # Cool down
        temperature *= cooling_rate
        revenue_history.append(current_revenue)
        
        # Debug output every 2000 iterations
        if iteration % 2000 == 0:
            print(f"  Iteration {iteration}: Revenue=${current_revenue:,.2f}, Temp={temperature:.1f}")
    
    return {
        'optimal_prices': best_prices,
        'optimal_attendance': best_attendance,
        'max_revenue': best_revenue,
        'revenue_history': revenue_history,
        'iterations': len(revenue_history)
    }

print("Simulated annealing optimization function defined")

In [ ]:
# Run fast optimization
print("\n" + "="*60)
print("RUNNING FAST EXHAUSTIVE SEARCH")
print("="*60)
fast_results = optimize_pricing_fast(
    model=model,
    base_game_features=base_game_features,
    price_ranges=price_ranges,
    capacity_limits=capacity_limits
)

In [ ]:
# simulated annealing optimization
print("\n" + "="*60)
print("RUNNING SIMULATED ANNEALING OPTIMIZATION")
print("="*60)
annealing_results = optimize_pricing_random_search(
    model=model,
    base_game_features=base_game_features,
    price_ranges=price_ranges,
    capacity_limits=capacity_limits,
    n_iterations=10000,  # Adjust as needed
    initial_temp=1000,  # Starting temperature
    cooling_rate=0.995  # Cooling rate for simulated annealing
)

In [ ]:

# Compare results
print("\n" + "="*80)
print("COMPARISON OF OPTIMIZATION METHODS")
print("="*80)

print("\nFAST EXHAUSTIVE SEARCH RESULTS:")
print("-" * 40)
for section, price in fast_results['optimal_prices'].items():
    section_name = section.replace('_price', '').replace('_', ' ').title()
    print(f"  {section_name:15}: ${price:4d}")
print(f"Maximum Revenue: ${fast_results['max_revenue']:,.2f}")
print(f"Combinations tested: {fast_results['total_combinations_tested']:,}")

print("\nSIMULATED ANNEALING OPTIMIZATION RESULTS:")
print("-" * 40)
for section, price in annealing_results['optimal_prices'].items():
    section_name = section.replace('_price', '').replace('_', ' ').title()
    print(f"  {section_name:15}: ${price:7.1f}")
print(f"Maximum Revenue: ${annealing_results['max_revenue']:,.2f}")
print(f"Iterations: {annealing_results['iterations']}")
# Revenue difference
revenue_diff = abs(fast_results['max_revenue'] - annealing_results['max_revenue'])
revenue_pct = (revenue_diff / fast_results['max_revenue']) * 100

print(f"\nRevenue Difference: ${revenue_diff:,.2f} ({revenue_pct:.3f}%)")

if annealing_results['max_revenue'] > fast_results['max_revenue']:
    print("✓ Simulated annealing found higher revenue (may need finer luxury box increments)")
elif abs(revenue_diff) < 100:
    print("✓ Both methods converged to similar results")
else:
    print("✓ Exhaustive search found higher revenue (global optimum)")

## 15. Interactive Price and Attendance Explorer

Use the interactive widgets below to explore how different pricing strategies affect predicted attendance and revenue for each seating section.

In [ ]:
# Interactive pricing explorer with real-time updates (using HBox/VBox layout)
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

class PricingExplorer:
    def __init__(self):
        # Create sliders
        self.bleachers_slider = widgets.FloatSlider(
            value=12.5, min=5, max=20, step=0.5,
            description='Bleachers ($):',
            style={'description_width': '120px'},
            layout=widgets.Layout(width='400px')
        )
        
        self.lower_tier_slider = widgets.FloatSlider(
            value=44.0, min=18, max=70, step=1.0,
            description='Lower Tier ($):',
            style={'description_width': '120px'},
            layout=widgets.Layout(width='400px')
        )
        
        self.courtside_slider = widgets.FloatSlider(
            value=125.0, min=50, max=200, step=5.0,
            description='Courtside ($):',
            style={'description_width': '120px'},
            layout=widgets.Layout(width='400px')
        )
        
        self.luxury_boxes_slider = widgets.FloatSlider(
            value=1000.0, min=400, max=1600, step=25.0,
            description='Luxury Boxes ($):',
            style={'description_width': '120px'},
            layout=widgets.Layout(width='400px')
        )
        
        # Create output widget for plots
        self.plot_output = widgets.Output()
        
        # Create output widget for table
        self.table_output = widgets.Output()
        
        # Initial update
        self._update_display()
        
        # Link sliders to update function (but only after initial setup)
        self.bleachers_slider.observe(self._on_change, names='value')
        self.lower_tier_slider.observe(self._on_change, names='value')
        self.courtside_slider.observe(self._on_change, names='value')
        self.luxury_boxes_slider.observe(self._on_change, names='value')
    
    def _on_change(self, change):
        """Handle slider changes"""
        self._update_display()
    
    def _update_display(self):
        """Update the display with current slider values"""
        # Get current values
        bleachers_price = self.bleachers_slider.value
        lower_tier_price = self.lower_tier_slider.value
        courtside_price = self.courtside_slider.value
        luxury_boxes_price = self.luxury_boxes_slider.value
        
        # Clear previous outputs
        self.plot_output.clear_output(wait=True)
        self.table_output.clear_output(wait=True)
        
        # Create game features
        current_prices = {
            'bleachers_price': bleachers_price,
            'lower_tier_price': lower_tier_price,
            'courtside_price': courtside_price,
            'luxury_boxes_price': luxury_boxes_price
        }
        
        game_features = base_game_features.copy()
        game_features.update(current_prices)
        
        try:
            # Make prediction
            predicted_attendance = predict_attendance(model, game_features)
            
            # Calculate revenue
            total_revenue = (
                predicted_attendance['bleachers'] * bleachers_price +
                predicted_attendance['lower_tier'] * lower_tier_price +
                predicted_attendance['courtside'] * courtside_price +
                predicted_attendance['luxury_boxes'] * luxury_boxes_price
            )
            
            # Update plots
            with self.plot_output:
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
                
                # Attendance bar chart
                sections = ['Bleachers', 'Lower Tier', 'Courtside', 'Luxury Boxes']
                attendances = [
                    predicted_attendance['bleachers'],
                    predicted_attendance['lower_tier'], 
                    predicted_attendance['courtside'],
                    predicted_attendance['luxury_boxes']
                ]
                colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
                
                bars1 = ax1.bar(sections, attendances, color=colors, alpha=0.7)
                ax1.set_title('Predicted Attendance by Section', fontsize=14, fontweight='bold')
                ax1.set_ylabel('Attendance (seats)', fontsize=12)
                ax1.grid(True, alpha=0.3)
                
                # Add value labels on bars
                for bar, attendance in zip(bars1, attendances):
                    height = bar.get_height()
                    ax1.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                            f'{attendance:,}', ha='center', va='bottom', fontweight='bold')
                
                # Revenue bar chart
                revenues = [
                    predicted_attendance['bleachers'] * bleachers_price,
                    predicted_attendance['lower_tier'] * lower_tier_price,
                    predicted_attendance['courtside'] * courtside_price,
                    predicted_attendance['luxury_boxes'] * luxury_boxes_price
                ]
                
                bars2 = ax2.bar(sections, revenues, color=colors, alpha=0.7)
                ax2.set_title('Revenue by Section', fontsize=14, fontweight='bold')
                ax2.set_ylabel('Revenue ($)', fontsize=12)
                ax2.grid(True, alpha=0.3)
                
                # Add value labels on bars
                for bar, revenue in zip(bars2, revenues):
                    height = bar.get_height()
                    ax2.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                            f'${revenue:,.0f}', ha='center', va='bottom', fontweight='bold')
                
                plt.tight_layout()
                plt.show()
            
            # Update table
            with self.table_output:
                print("="*60)
                print("PRICING STRATEGY SUMMARY")
                print("="*60)
                print(f"{'Section':<15} {'Price ($)':<12} {'Attendance':<12} {'Revenue ($)':<15}")
                print("-" * 60)
                
                section_names = ['Bleachers', 'Lower Tier', 'Courtside', 'Luxury Boxes']
                prices = [bleachers_price, lower_tier_price, courtside_price, luxury_boxes_price]
                
                for section, price, attendance, revenue in zip(section_names, prices, attendances, revenues):
                    print(f"{section:<15} ${price:<11.1f} {attendance:<12,} ${revenue:<14,.0f}")
                
                print("-" * 60)
                print(f"{'TOTAL':<15} {'':<12} {sum(attendances):<12,} ${total_revenue:<14,.0f}")
                print("="*60)
                
        except Exception as e:
            with self.table_output:
                print(f"Error making prediction: {e}")
    
    def display(self):
        """Display the interactive explorer"""
        # Create layout
        sliders_box = widgets.VBox([
            widgets.HTML("<h3>Interactive Pricing Explorer</h3>"),
            widgets.HTML("<p>Adjust the sliders below to see real-time updates:</p>"),
            self.bleachers_slider,
            self.lower_tier_slider,
            self.courtside_slider,
            self.luxury_boxes_slider
        ])
        
        # Display everything
        display(sliders_box, self.plot_output, self.table_output)

# Create and display the explorer
print("Creating Interactive Pricing Explorer...")
explorer = PricingExplorer()
explorer.display()